In [28]:
#install stuff
# IMPORTANT NOTE: before starting Jupyter Notebook, navigate to /home/peter/Desktop/yolov5. Do source .venv/bin/activate
# this sets up the virtual environment where torch is installed and allows yolo to work.
# if installing this fresh, you'll need to install torch with Cuda (or not depending on the hardware)
#Yolov5
%cd /home/peter/Desktop/yolov5
! source .venv/bin/activate
pip install ultralytics
pip install seaborn
#pip install python>=3.1.30
!pip install matplotlib>=3.3
!pip install numpy>=1.23.5
!pip install opencv-python>=4.1.1
!pip install pillow>=10.3.0
!pip install psutil  # system resources
!pip install PyYAML>=5.3.1
!pip install requests>=2.32.2
!pip install scipy>=1.4.1
!pip install hop>=0.1.1  # FLOPs computation
!pip install torch>=1.8.0  # see https://pytorch.org/get-started/locally (recommended)
!pip install torchvision>=0.9.0
!pip install tqdm>=4.66.3
!pip install ultralytics>=8.2.64  # https://ultralytics.com
!pip install pandas>=1.1.4
!pip install seaborn>=0.11.0
# assume the files are in the right places. See https://github.com/ch-hristov/p-id-symbols for details. Follow the notebook
#Ollama
!pip install ollama
#Claude
!pip install anthropic


SyntaxError: invalid syntax (55929743.py, line 9)

In [1]:
#yolo
import pandas as pd
import numpy as np
from PIL import Image
import ast
import os
from pathlib import Path
import shutil

import argparse
import csv
import platform
import sys
# import torch ***this one is trouble

#ollama
import ollama
import base64
# import os duplicate
import glob
from pathlib import Path
import json
from typing import List, Dict, Optional

#Claude
import anthropic

#image handling
import os
import base64
from pathlib import Path
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import fitz  # PyMuPDF
import io


In [2]:
#Anthropic key - do not publish
key="sk-ant-api03redactedswAA"

order of operations
1 Split the PDF into 4 quadrants
2 Get the metadata from the titleblock in the bottom right quadrant
3 Get the line list from a Claude query into a JSON string
4 Get the valve list from Yolov5
5 Go through each valve in the valve list and get the size and tag number via a png snippet around the valve. Include the coordinates of valve (include an offset for the quadrant). 
6 Get the line number of the valve, if possible, and add it to the valve list.
7 Get an instrument list using the data from Yolov5. Get the tag name/number for each instrument and coordinates.
8 Get an equipment list from a LLM (may need to use Claude)
8 Use one of the LLMs to create a DEXPI file from the metatdata, line list, valve list, instrument list, and equipment list.


In [3]:
#Split P&ID into 4 quadrants

def pdf_to_images(pdf_path, page_number=0, dpi=300):
    """
    Convert a PDF page to a high-quality PIL Image
    
    Args:
        pdf_path (str): Path to the PDF file
        page_number (int): Page number to convert (0-indexed)
        dpi (int): Resolution for the conversion
    
    Returns:
        PIL.Image: The converted page as an image
    """
    doc = fitz.open(pdf_path)
    
    if page_number >= len(doc):
        raise ValueError(f"Page {page_number} doesn't exist. PDF has {len(doc)} pages.")
    
    page = doc[page_number]
    
    # Create a transformation matrix for higher resolution
    mat = fitz.Matrix(dpi/72, dpi/72)  # 72 is the default DPI
    
    # Render the page as a pixmap
    pix = page.get_pixmap(matrix=mat)
    
    # Convert to PIL Image
    img_data = pix.tobytes("png")
    image = Image.open(io.BytesIO(img_data))
    
    doc.close()
    return image

def split_image_into_quadrants(image):
    """
    Split a PIL Image into 4 equal quadrants
    
    Args:
        image (PIL.Image): The source image
    
    Returns:
        tuple: Four PIL Images representing the quadrants (top-left, top-right, bottom-left, bottom-right)
    """
    width, height = image.size
    
    # Calculate quadrant dimensions
    half_width = width // 2
    half_height = height // 2
    
    # Define crop boxes for each quadrant
    # (left, top, right, bottom)
    top_left = (0, 0, half_width, half_height)
    top_right = (half_width, 0, width, half_height)
    bottom_left = (0, half_height, half_width, height)
    bottom_right = (half_width, half_height, width, height)
    
    # Crop the image into quadrants
    quadrants = (
        image.crop(top_left),
        image.crop(top_right),
        image.crop(bottom_left),
        image.crop(bottom_right)
    )
    
    return quadrants

def save_quadrants_as_png(quadrants, output_dir, base_filename):
    """
    Save quadrant images as PNG files
    
    Args:
        quadrants (tuple): Four PIL Images
        output_dir (str): Directory to save the images
        base_filename (str): Base name for the output files
    
    Returns:
        list: Paths to the saved PNG files
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    quadrant_names = ["top_left", "top_right", "bottom_left", "bottom_right"]
    saved_paths = []
    
    for i, (quadrant, name) in enumerate(zip(quadrants, quadrant_names)):
        filename = f"{base_filename}_q{i+1}.png"
#        filename = f"{base_filename}_quadrant_{i+1}_{name}.png"
        filepath = output_dir / filename
        
        # Save as PNG with high quality
        quadrant.save(filepath, "PNG", optimize=True)
        saved_paths.append(str(filepath))
#        print(f"Saved quadrant {i+1} ({name}): {filepath}")
    
    return saved_paths
    
def create_quads(pdf_path,page,output_dir):
     image = pdf_to_images(pdf_path, page)
     quadrants = split_image_into_quadrants(image)
# Save quadrants as PNG files
     base_filename = Path(pdf_path).stem
     quadrant_paths = save_quadrants_as_png(quadrants, output_dir, base_filename)


In [4]:
# call Ollama LLMs. Encode PNGs too.
def encode_png_file(png_path: str) -> Optional[str]:
    """
    Encode a PNG file to base64
    
    Args:
        png_path (str): Path to the PNG file
    
    Returns:
        str: Base64 encoded image data, or None if error
    """
    try:
        with open(png_path, 'rb') as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding {png_path}: {e}")
        return None

def find_png_files(directory: str, pattern: str = "*.png") -> List[str]:
    """
    Find all PNG files in a directory
    
    Args:
        directory (str): Directory to search
        pattern (str): File pattern (default: "*.png")
    
    Returns:
        List[str]: List of PNG file paths
    """
    search_path = os.path.join(directory, pattern)
    png_files = glob.glob(search_path)
    png_files.sort()  # Sort for consistent ordering
    return png_files

def query_ollama(model, prompt, png_path):
    if png_path != "": 
        encoded_image = encode_png_file(png_path)
    
        response = ollama.chat(
           model=model,
           messages=[{'role': 'user', 'content': prompt,'images': [encoded_image]}],
           options={
               'temperature': 0.7,  # Range: 0.0 to 1.0
           }
        )
    else:
        response = ollama.chat(
           model=model,
           messages=[{'role': 'user', 'content': prompt}],
           options={
               'temperature': 0.7,  # Range: 0.0 to 1.0
           }
        )

    foo = response.message.content[8:-4]
    #print(response.message.content)
    response_J = json.loads(foo)
    return response_J

def query_ollama_multiple(model, prompt, png_path): #same as above, except png_path is a list of pngs
    #!!!!!This does not work. Right now ollama does not support multiple images
    encoded_images = []
    
    for i,png in enumerate(png_path):
       encoded_images.append(encode_png_file(png))
   
    response = ollama.chat(
       model=model,
       messages=[{'role': 'user', 'content': prompt,'images': [encoded_images]}],
       options={
           'temperature': 0.7,  # Range: 0.0 to 1.0
       }
    )

    foo = response.message.content[8:-4]
    response_J = json.loads(foo)
    return response_J    

In [5]:
#Claude
Claude = "claude-sonnet-4-20250514"

#initialize
client = anthropic.Anthropic(
    api_key=key  # or set ANTHROPIC_API_KEY environment variable
)

def claude_simple(prompt_text):
    content = []
    content.append({
        "type": "text",
        "text": prompt_text
    })
    
    message = client.messages.create(
        model=Claude,
        max_tokens=2000,
        temperature = 0.5,
        messages=[{
            "role": "user",
            "content": content
        }]
    )
    return message.content[0].text 

def upload_multiple_images_to_claude(image_paths, prompt_text):
    """
    Upload multiple images to Claude
    
    Args:
        image_paths (list): List of paths to image files
        prompt_text (str): Text prompt to send along with the images
    
    Returns:
        str: Claude's response
    """
    
    content = []
    
    # Add all images
    for i, image_path in enumerate(image_paths):
        image_path = Path(image_path)
        
        with open(image_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode('utf-8')
        
        content.append({
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": image_data
            }
        })
    
    # Add text prompt
    content.append({
        "type": "text",
        "text": prompt_text
    })
    
    message = client.messages.create(
        model=Claude,
        max_tokens=2000,
        temperature = 0.5,
        messages=[{
            "role": "user",
            "content": content
        }]
    )
    
    return message.content[0].text

In [41]:
# Cropping YOLO predictions
def crop_predictions(png_file_path, predictions_csv_path, scaling_factor=1.0, output_dir="cropped_images"):
    """
    Crop image regions based on predictions CSV with bounding box coordinates.
    
    Parameters:
    png_file_path (str): Path to the input PNG image
    predictions_csv_path (str): Path to the predictions CSV file
    scaling_factor (float): Factor to scale the bounding box size (1.0 = original size)
    output_dir (str): Directory to save cropped images
    """
    
    # Read the CSV file
    df = pd.read_csv(predictions_csv_path)
    
    # Load the image
    image = Image.open(png_file_path)
    image_width, image_height = image.size
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Get the base name of the PNG file (without extension)
    base_name = Path(png_file_path).stem
    
    cropped_images = []
    
    for idx, row in df.iterrows():
        # Parse the xyxy coordinates from string representation
        # Remove 'tensor()' wrapper and convert to float
        coords_str = row['Xyxy'].strip('[]')
        coords_parts = coords_str.split(', ')
        
        # Extract numerical values from tensor format
        coords = []
        for part in coords_parts:
            # Remove 'tensor(' and ')' and convert to float
            num_str = part.strip().replace('tensor(', '').replace(')', '').replace('.', '')
            coords.append(float(num_str))
        
        x1, y1, x2, y2 = coords
        
        # Calculate center and dimensions of bounding box
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1
        
        # Apply scaling factor
        scaled_width = width * scaling_factor
        scaled_height = height * scaling_factor
        
        # Calculate new coordinates
        new_x1 = center_x - scaled_width / 2
        new_y1 = center_y - scaled_height / 2
        new_x2 = center_x + scaled_width / 2
        new_y2 = center_y + scaled_height / 2
        
        # Ensure coordinates are within image bounds
        new_x1 = max(0, min(new_x1, image_width))
        new_y1 = max(0, min(new_y1, image_height))
        new_x2 = max(0, min(new_x2, image_width))
        new_y2 = max(0, min(new_y2, image_height))
        
        # Crop the image
        cropped_img = image.crop((new_x1, new_y1, new_x2, new_y2))
        
        # Create filename for cropped image
        prediction_clean = row['Prediction']
        confidence = f"{row['Confidence']:.2f}"
        output_filename = f"{base_name}_{idx:03d}_{prediction_clean}_conf{confidence}.png"
        output_path = os.path.join(output_dir, output_filename)
        
        # Save cropped image
        cropped_img.save(output_path)
        
        cropped_images.append({
            'index': idx,
            'prediction': row['Prediction'],
            'confidence': row['Confidence'],
            'original_coords': (x1, y1, x2, y2),
            'scaled_coords': (new_x1, new_y1, new_x2, new_y2),
            'output_path': output_path,
            'size': cropped_img.size
        })
        
        print(f"Saved: {output_filename} ({cropped_img.size[0]}x{cropped_img.size[1]})")
    
    print(f"\nProcessed {len(cropped_images)} predictions")
    print(f"Cropped images saved to: {output_dir}")
    
    return cropped_images


In [49]:
def class_lookup(prediction):
    match prediction:
        case 1:
            res = "Gate_Valve"
        case 2:
            res = "Ball_Valve"
        case 3:
            res = "Globe_valve_NO"
        case 4: 
            res = "Gate_valve_NO"
        case 5: 
            res = "Globe_valve_NO"
        case 6: 
            res = "Butterfly_Valve"
        case 7: 
            res = "Plug Valve"
        case 8: 
            res = "Check_Valve"
        case 9: 
            res = "Diaphragm_valve"
        case 10: 
            res = "Needle_valve"
        case 11: 
            res = "Half_Filled_Gate_Valve"
        case 12: 
            res = "Gate_Valve_NC"
        case 13: 
            res = "Globle_valve_NC"
        case 14: 
            res = "Control_Valve"
        case 15:
            res = "Rotary_Valve"
        case 16: 
            res = "Ball_valve_NC"
# probably should do the rest too, but enough for now...
    return res

""" classes
   1: Gate_Valve
    2: Ball_Valve
    3: Globe_valve_NO
    4: Gate_valve_NO
    5: Globe_valve_NO
    6: Butterfly_valve
    7: Plug valve
    8: Check_valve
    9: Diaphragm_valve
    10: Needle_valve
    11: Half_Filled_Gate_Valve
    12: Gate_Valve_NC
    13: Globle_valve_NC
    14: Control_Valve
    15: Rotary_Valve
    16: Ball_valve_NC
    17: Paddle_blind
    18: Spectacle_blind_Closed
    19: Spectacle_blind_Open
    20: Reducer
    21: Flange_or_Nozzle
    22: Rupture_disk
    23: Pipe_Insulation_or_Tracing
    24: Flow_Arrow
    25: sight_glass
    26: Instrument_Field
    27: Instrument_Field
    28: Instrument_Panel
    29: Instrument_Aux_Panel
    30: box
    31: Instrument_Panel
    32: box
    """

' classes\n   1: Gate_Valve\n    2: Ball_Valve\n    3: Globe_valve_NO\n    4: Gate_valve_NO\n    5: Globe_valve_NO\n    6: Butterfly_valve\n    7: Plug valve\n    8: Check_valve\n    9: Diaphragm_valve\n    10: Needle_valve\n    11: Half_Filled_Gate_Valve\n    12: Gate_Valve_NC\n    13: Globle_valve_NC\n    14: Control_Valve\n    15: Rotary_Valve\n    16: Ball_valve_NC\n    17: Paddle_blind\n    18: Spectacle_blind_Closed\n    19: Spectacle_blind_Open\n    20: Reducer\n    21: Flange_or_Nozzle\n    22: Rupture_disk\n    23: Pipe_Insulation_or_Tracing\n    24: Flow_Arrow\n    25: sight_glass\n    26: Instrument_Field\n    27: Instrument_Field\n    28: Instrument_Panel\n    29: Instrument_Aux_Panel\n    30: box\n    31: Instrument_Panel\n    32: box\n    '

In [73]:
def process_valves(valves,full_dwg,detect_path):
    for quad in range(4):
        #image size is important - if the size is too far off, detection is not accurate
        run_path = '"./detect.py" --weights "./best.pt" --source '+full_dwg[quad]+' --conf-thres 0.5 --save-csv --imgsz 5000'
        %run {run_path}
        result_path = detect_path+"/exp"
        #create cropped images
        crop_predictions(full_dwg[quad], result_path+"/predictions.csv", scaling_factor=10.0, output_dir=result_path)
        #go through each detected item and add the ones we want to the valve list
            # Read the CSV file
        df = pd.read_csv(result_path+"/predictions.csv")
        base_name = Path(full_dwg[quad]).stem
        for idx, row in df.iterrows():
            # Parse the xyxy coordinates from string representation
            # Remove 'tensor()' wrapper and convert to float
            coords_str = row['Xyxy'].strip('[]')
            coords_parts = coords_str.split(', ')
            prediction = row['Prediction']
            #Prediction holds an int containing the type of item found. We only want certain types
            if prediction > 0 and prediction <=16: # this is a valve
               classname = class_lookup(prediction)
               # Extract numerical values from tensor format to get the valve center coordinates
               coords = []
               for part in coords_parts:
               # Remove 'tensor(' and ')' and convert to float
                   num_str = part.strip().replace('tensor(', '').replace(')', '').replace('.', '')
                   coords.append(float(num_str))
               x1, y1, x2, y2 = coords
               # Calculate center and dimensions of bounding box
               center_x = (x1 + x2) / 2
               center_y = (y1 + y2) / 2
               #Get the image filename and send it to a LLM to get the size and tag number
               # reCreate filename for cropped image
               prediction_clean = row['Prediction']
               confidence = f"{row['Confidence']:.2f}"
               output_filename = f"{base_name}_{idx:03d}_{prediction_clean}_conf{confidence}.png"
               png_path = os.path.join(result_path, output_filename)
               prompt = """Look at the valve in the center of the image. Return the valve size and valve tag number as a 
               json string. If either value can't be found return the value as none"""
               result = query_ollama(ollama_model, prompt, png_path)
               print(result)
               valve_size = result[0]
               valve_tag = result[1]
#FUTURE: try to get the line number of the valve too
               valvedict = {"tag": valve_tag, "type": classname, "size": valve_size, "center_x": center_x, "center_y": center_y}
               valve.append(valvedict)
    return valves


In [74]:
#main
pdf_file = "/home/peter/test3.pdf"
page = 0
ollama_model = 'gemma3:27b'

output_dir = pdf_file[:-4]
filename = os.path.splitext(os.path.basename(pdf_file))[0]
create_quads(pdf_file,page,output_dir) #create PNG files

# Call Gemma3:27b via Ollama to get metadata
prompt = """List the project name, company name, drawing (DWG) name, drawing number, location, date, revision, author, 
and any other information found in the title block in the lower right corner of the image. Return data as JSON."""
png_path = output_dir+"/"+filename+"_q4.png"
drawing_info = query_ollama(ollama_model, prompt, png_path
                           )
# Call Claude and get line list. Need to send all 4 quadrants
full_dwg = [] #all 4 quadrants
for i in range(4):
    n = i+1
    full_dwg.append(output_dir+"/"+filename+"_q"+str(n)+".png")
prompt = """list the pipe lines on this P&ID with the line number, primary size, and what equipment, off-page connectors, 
or other lines it is connected to. Return the data as JSON and only return JSON."""
result = upload_multiple_images_to_claude(full_dwg, prompt)
line_list = json.loads(result[8:-4])

#equipment list
prompt = """List the equipment on this P&ID with the name, description and associated metadata. Only list the primary equipment. 
Do not list piping, valves, instruments or information about the drawing itself. Return data as JSON"""
result = upload_multiple_images_to_claude(full_dwg, prompt)
equip_list = json.loads(result[8:-4])

#valves
valves = []
#clean up previous runs
detect_path = "/home/peter/Desktop/yolov5/runs/detect"
shutil.rmtree(detect_path)
# go through each prediction and add the right info to the valve list
process_valves(valves,full_dwg,detect_path)



    


detect: weights=['./best.pt'], source=/home/peter/test3/test3_q1.png, data=data/coco128.yaml, imgsz=[5000, 5000], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 v7.0-430-g459d8bf0 Python-3.12.3 torch-2.8.0+cu128 CPU

Fusing layers... 
Model summary: 267 layers, 46280598 parameters, 0 gradients, 108.2 GFLOPs
WARNING ⚠️ --img-size [5000, 5000] must be multiple of max stride 32, updating to [5024, 5024]
image 1/1 /home/peter/test3/test3_q1.png: 3552x5024 7 4s, 9 12s, 3 13s, 2 19s, 1 20, 1 21, 9 23s, 9 24s, 3 26s, 2 32s, 8254.4ms
Speed: 14.4ms pre-process, 8254.4ms inference, 6.4ms NMS per image at shape (1, 3, 5024, 5024)
Results saved to runs/detect/exp


Saved: test3_q1_000_32_conf0.56.png (1160x530)
Saved: test3_q1_001_24_conf0.63.png (770x380)
Saved: test3_q1_002_13_conf0.63.png (149x410)
Saved: test3_q1_003_24_conf0.64.png (410x610)
Saved: test3_q1_004_4_conf0.64.png (830x450)
Saved: test3_q1_005_26_conf0.65.png (340x400)
Saved: test3_q1_006_26_conf0.66.png (1570x1630)
Saved: test3_q1_007_21_conf0.67.png (290x160)
Saved: test3_q1_008_23_conf0.68.png (1170x500)
Saved: test3_q1_009_26_conf0.68.png (1560x1201)
Saved: test3_q1_010_24_conf0.69.png (730x420)
Saved: test3_q1_011_24_conf0.72.png (400x680)
Saved: test3_q1_012_24_conf0.72.png (670x600)
Saved: test3_q1_013_24_conf0.72.png (670x610)
Saved: test3_q1_014_13_conf0.73.png (210x420)
Saved: test3_q1_015_24_conf0.75.png (400x650)
Saved: test3_q1_016_32_conf0.75.png (1190x520)
Saved: test3_q1_017_4_conf0.75.png (810x440)
Saved: test3_q1_018_23_conf0.77.png (1170x500)
Saved: test3_q1_019_20_conf0.82.png (460x780)
Saved: test3_q1_020_12_conf0.82.png (420x780)
Saved: test3_q1_021_24_conf0

KeyError: 0

In [72]:

valves = []
#clean up previous runs
detect_path = "/home/peter/Desktop/yolov5/runs/detect"
shutil.rmtree(detect_path)

process_valves(valves,full_dwg,detect_path)
print(valves)

detect: weights=['./best.pt'], source=/home/peter/test3/test3_q1.png, data=data/coco128.yaml, imgsz=[5000, 5000], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 v7.0-430-g459d8bf0 Python-3.12.3 torch-2.8.0+cu128 CPU

Fusing layers... 
Model summary: 267 layers, 46280598 parameters, 0 gradients, 108.2 GFLOPs
WARNING ⚠️ --img-size [5000, 5000] must be multiple of max stride 32, updating to [5024, 5024]
image 1/1 /home/peter/test3/test3_q1.png: 3552x5024 7 4s, 9 12s, 3 13s, 2 19s, 1 20, 1 21, 9 23s, 9 24s, 3 26s, 2 32s, 8921.0ms
Speed: 19.1ms pre-process, 8921.0ms inference, 15.9ms NMS per image at shape (1, 3, 5024, 5024)
Results saved to runs/detect/exp


Saved: test3_q1_000_32_conf0.56.png (1160x530)
Saved: test3_q1_001_24_conf0.63.png (770x380)
Saved: test3_q1_002_13_conf0.63.png (149x410)
Saved: test3_q1_003_24_conf0.64.png (410x610)
Saved: test3_q1_004_4_conf0.64.png (830x450)
Saved: test3_q1_005_26_conf0.65.png (340x400)
Saved: test3_q1_006_26_conf0.66.png (1570x1630)
Saved: test3_q1_007_21_conf0.67.png (290x160)
Saved: test3_q1_008_23_conf0.68.png (1170x500)
Saved: test3_q1_009_26_conf0.68.png (1560x1201)
Saved: test3_q1_010_24_conf0.69.png (730x420)
Saved: test3_q1_011_24_conf0.72.png (400x680)
Saved: test3_q1_012_24_conf0.72.png (670x600)
Saved: test3_q1_013_24_conf0.72.png (670x610)
Saved: test3_q1_014_13_conf0.73.png (210x420)
Saved: test3_q1_015_24_conf0.75.png (400x650)
Saved: test3_q1_016_32_conf0.75.png (1190x520)
Saved: test3_q1_017_4_conf0.75.png (810x440)
Saved: test3_q1_018_23_conf0.77.png (1170x500)
Saved: test3_q1_019_20_conf0.82.png (460x780)
Saved: test3_q1_020_12_conf0.82.png (420x780)
Saved: test3_q1_021_24_conf0

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [64]:
print (full_dwg)

['/home/peter/test3/test3_q1.png', '/home/peter/test3/test3_q2.png', '/home/peter/test3/test3_q3.png', '/home/peter/test3/test3_q4.png']
